# dynamo db - create table

In [15]:
import os
import json
import time
import boto3
import tweepy
import pprint
import datetime
import numpy as np
import pandas as pd
import botocore.session
from boto3.dynamodb.conditions import Key, Attr

with open("../creds.json", "r") as f:
    creds = json.load(f)
    f.close()
    
print("imported modules.")

imported modules.


## dynamodb client

In [16]:
client = boto3.client('dynamodb',
  endpoint_url = "http://192.168.50.114:8000",
  aws_access_key_id=creds["aws-access-key"],
  aws_secret_access_key=creds["aws-secret-access-key"],
  region_name='us-east-2')

dynamodb = boto3.resource('dynamodb', \
            endpoint_url = "http://192.168.50.114:8000", \
            region_name='us-east-2')

print("created dynamo client.")

created dynamo client.


## create dynamo table

In [17]:
table = dynamodb.create_table(
    TableName='tweets',
    KeySchema=[
        {
            'AttributeName': 'user',
            'KeyType': 'HASH'  #Partition key
        },
        {
            'AttributeName': 'created_at',
            'KeyType': 'RANGE'  #Sort key
        }
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'user',
            'AttributeType': 'S'
        },
        {
            'AttributeName': 'created_at',
            'AttributeType': 'S'
        }

    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 10,
        'WriteCapacityUnits': 10
    }
)

print("Table status:", table.table_status)

Table status: ACTIVE


## wait for table creation

In [19]:
session = botocore.session.get_session()
dynamodb_session = session.create_client('dynamodb', \
                    region_name='us-east-2',
                    endpoint_url = "http://192.168.50.114:8000",
                    aws_access_key_id=creds["aws-access-key"],
                    aws_secret_access_key=creds["aws-secret-access-key"]) # low-level client

waiter = dynamodb_session.get_waiter('table_exists')
waiter.wait(TableName="tweets")

KeyboardInterrupt: 

In [20]:
print("Table status:", table.table_status)

Table status: ACTIVE


## scan tweets table

In [21]:
tweets = dynamodb.Table("tweets")

response = tweets.scan()

In [22]:
response

{'Items': [],
 'Count': 0,
 'ScannedCount': 0,
 'ResponseMetadata': {'RequestId': '7bb01f1f-09e4-4d8a-8d33-9fd7b41d53c1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sat, 04 Feb 2023 19:35:19 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '583912891',
   'x-amzn-requestid': '7bb01f1f-09e4-4d8a-8d33-9fd7b41d53c1',
   'content-length': '39',
   'server': 'Jetty(9.4.18.v20190429)'},
  'RetryAttempts': 0}}

In [23]:
! ifconfig wlx984827c9c1b7

wlx984827c9c1b7: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 1500
        inet 192.168.50.114  netmask 255.255.255.0  broadcast 192.168.50.255
        inet6 fe80::6162:562d:4e76:6ce7  prefixlen 64  scopeid 0x20<link>
        ether 98:48:27:c9:c1:b7  txqueuelen 1000  (Ethernet)
        RX packets 594978  bytes 7687847678 (7.6 GB)
        RX errors 0  dropped 458393  overruns 0  frame 0
        TX packets 320059  bytes 2057236482 (2.0 GB)
        TX errors 0  dropped 31 overruns 0  carrier 0  collisions 0



In [24]:
! aws dynamodb list-tables --endpoint-url http://192.168.50.114:8000

{
    "TableNames": [
        "tweets"
    ]
}


In [25]:
! aws dynamodb describe-table --table-name "tweets" --endpoint-url http://192.168.50.114:8000

{
    "Table": {
        "AttributeDefinitions": [
            {
                "AttributeName": "user",
                "AttributeType": "S"
            },
            {
                "AttributeName": "created_at",
                "AttributeType": "S"
            }
        ],
        "TableName": "tweets",
        "KeySchema": [
            {
                "AttributeName": "user",
                "KeyType": "HASH"
            },
            {
                "AttributeName": "created_at",
                "KeyType": "RANGE"
            }
        ],
        "TableStatus": "ACTIVE",
        "CreationDateTime": 1675538770.303,
        "ProvisionedThroughput": {
            "LastIncreaseDateTime": 0.0,
            "LastDecreaseDateTime": 0.0,
            "NumberOfDecreasesToday": 0,
            "ReadCapacityUnits": 10,
            "WriteCapacityUnits": 10
        },
        "TableSizeBytes": 0,
        "ItemCount": 0,
        "TableArn": "arn:aws:dynamodb:ddblocal:000000000000:table/t